## Explore the impact of the number of steps per period on the learning

In [ ]:
# define parameters configuration
#     plt.close('all')
#     env_seed = 123
#     total_num_timesteps = 60000
#     num_periods = 2000
#     TRAINING_KWARGS['num_periods'] = num_periods
#     # create folder to save data based on env seed
#     main_folder = 'C:/Users/saraf/OneDrive/Documentos/IDIBAPS/foraging RNNs/nets/'
#    # main_folder = '/home/molano/foragingRNNs_data/nets/'
#     # Set up the task
#     w_factor = 0.00001
#     mean_ITI = 200
#     max_ITI = 400
#     fix_dur = 100
#     dec_dur = 100
#     blk_dur = 50
#     probs = np.array([0.1, 0.9])
# define parameter to explore
# seq_len_mat = np.array([10, 20, 30, 40, 50, 60, 70, 80, 90, 100])
# for seq_len in seq_len_mat:
#   num_periods = total_num_timesteps // seq_len
# plot performance
